#Run fine-tuning job

In [1]:
import openai

with open('../credentials.txt', encoding = 'utf-8') as f:
    organization = f.readline().strip()
    api_key = f.readline().strip()
    
client = openai.OpenAI(organization=organization, api_key=api_key)

import pprint

In [21]:
response = client.files.create(
  file=open("train_4.jsonl", "rb"),
  purpose="fine-tune"
)

In [22]:
pprint.pprint(response)


FileObject(id='file-6mthb4eBqWjB9CksCuyYFC', bytes=413227, created_at=1736283384, filename='train_4.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [23]:
response2 = client.files.create(
  file=open("val_4.jsonl", "rb"),
  purpose="fine-tune"
)

In [24]:
pprint.pprint(response2)

FileObject(id='file-6yLWtEN351ZquQYfiGC1y5', bytes=112024, created_at=1736283391, filename='val_4.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [38]:
job = client.fine_tuning.jobs.create(
    training_file='file-6mthb4eBqWjB9CksCuyYFC',
    validation_file='file-6yLWtEN351ZquQYfiGC1y5',
    suffix = 'fold4',
    model="gpt-4o-mini-2024-07-18"
)
job  

FineTuningJob(id='ftjob-srsB9IEdxVIroirBgKlsSmIa', created_at=1736285454, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-aw09ch9csbZw51lKeR83xQAq', result_files=[], seed=550404552, status='validating_files', trained_tokens=None, training_file='file-6mthb4eBqWjB9CksCuyYFC', validation_file='file-6yLWtEN351ZquQYfiGC1y5', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix='fold4')

In [45]:
events = client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-srsB9IEdxVIroirBgKlsSmIa", limit=500)
pprint.pprint(events)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-cosmB0s0xrpamW8CPTqOg3FR', created_at=1736286521, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-N2y2ndLJnPTHf1Hvlkcl9IsP', created_at=1736286514, level='info', message='New fine-tuned model created', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-b3spBu4mhEz9zemheX1NSmUR', created_at=1736286514, level='info', message='Checkpoint created at step 414', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-sHvfyIhtU8YGOTBf9E4iBUiQ', created_at=1736286514, level='info', message='Checkpoint created at step 207', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-2bth2sNnTCLAO3ekUpBXZa4v', created_at=1736286461, level='info', message='Step 621/621: training loss=0.98, validation loss=1.64, full validation l

In [46]:
# Function to format and print events
def format_events(events):
    for event in events.data:
        print(f"Message: {event.message}")
        if event.data:
            print("Metrics:")
            for key, value in event.data.items():
                print(f"  {key}: {value}")

format_events(events)

Message: The job has successfully completed
Message: New fine-tuned model created
Message: Checkpoint created at step 414
Message: Checkpoint created at step 207
Message: Step 621/621: training loss=0.98, validation loss=1.64, full validation loss=2.14
Metrics:
  step: 621
  train_loss: 0.9844347834587097
  valid_loss: 1.6422446230624586
  total_steps: 621
  full_valid_loss: 2.135340309957924
  train_mean_token_accuracy: 0.7177419066429138
  valid_mean_token_accuracy: 0.723404255319149
  full_valid_mean_token_accuracy: 0.545315837656393
Message: Step 620/621: training loss=1.19, validation loss=1.48
Metrics:
  step: 620
  train_loss: 1.1945390701293945
  valid_loss: 1.4831044591706375
  total_steps: 621
  train_mean_token_accuracy: 0.7127071619033813
  valid_mean_token_accuracy: 0.6637931034482759
Message: Step 619/621: training loss=0.41
Metrics:
  step: 619
  train_loss: 0.4103790819644928
  total_steps: 621
  train_mean_token_accuracy: 0.9142857193946838
Message: Step 618/621: train

In [33]:
detail = client.fine_tuning.jobs.retrieve("ftjob-h55W7U720wbURqnTwFTSg547")
detail

FineTuningJob(id='ftjob-h55W7U720wbURqnTwFTSg547', created_at=1736283430, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-aw09ch9csbZw51lKeR83xQAq', result_files=[], seed=16678065, status='running', trained_tokens=None, training_file='file-6mthb4eBqWjB9CksCuyYFC', validation_file='file-6yLWtEN351ZquQYfiGC1y5', estimated_finish=1736286948, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix='fold4')

In [60]:
client.fine_tuning.jobs.retrieve("ftjob-NNHA92xUdDEyJHPWysrvXlyW")

FineTuningJob(id='ftjob-NNHA92xUdDEyJHPWysrvXlyW', created_at=1736142422, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:tedunderwood:fold1:AmaWUOgb', finished_at=1736143500, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-aw09ch9csbZw51lKeR83xQAq', result_files=['file-LbF6eitev3fk8EKFUsQueJ'], seed=465875889, status='succeeded', trained_tokens=256119, training_file='file-G5hQ6ECGsx981bMKH5Txny', validation_file='file-DMnvsN3abZhu3huz5fUYGD', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix='fold1')

In [72]:
result_file_id = "file-LbF6eitev3fk8EKFUsQueJ"
file_info = client.files.retrieve(result_file_id)
print(file_info)

FileObject(id='file-LbF6eitev3fk8EKFUsQueJ', bytes=21216, created_at=1736143504, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None)


In [2]:
file_content = client.files.download(file_id)

# Save the content to a local file
with open("result_file_fold1.jsonl", "wb") as file:
    file.write(file_content)

AttributeError: 'Files' object has no attribute 'download'